In [ ]:
import json
from collections import defaultdict, Counter
import os
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, hamming_loss, accuracy_score

# --- 1. Hàm đọc dữ liệu ASCA JSON (Giữ nguyên) ---
def load_asca_data(filepath):
    """Đọc dữ liệu từ file JSON định dạng ASCA."""
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            data = json.load(f)
            if "sentences" in data and isinstance(data["sentences"], list):
                return data["sentences"]
            else:
                print(f"Lỗi: Cấu trúc JSON không đúng trong file {filepath}.")
                return []
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file {filepath}")
        return []
    except json.JSONDecodeError:
        print(f"Lỗi: File {filepath} không phải là định dạng JSON hợp lệ.")
        return []
    except Exception as e:
        print(f"Lỗi không xác định khi đọc file: {e}")
        return []

# --- 2. Hàm huấn luyện và đánh giá mô hình đa nhãn (Giữ nguyên) ---
def train_evaluate_multilabel(X_train, y_train, X_test, y_test, labels_list, task_name=""):
    """Huấn luyện và đánh giá mô hình đa nhãn (TF-IDF + OneVsRest)."""
    print(f"\n--- Bắt đầu {task_name} ---")

    # Lựa chọn base classifier
    # base_classifier = LogisticRegression(solver='liblinear', random_state=42, class_weight='balanced')
    base_classifier = LinearSVC(random_state=42, dual="auto", class_weight='balanced')

    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(min_df=3, max_df=0.9, ngram_range=(1, 2))),
        ('clf', OneVsRestClassifier(base_classifier, n_jobs=-1))
    ])

    print("  Bắt đầu huấn luyện...")
    pipeline.fit(X_train, y_train)
    print("  Huấn luyện hoàn tất.")

    print("  Bắt đầu dự đoán...")
    y_pred = pipeline.predict(X_test)
    print("  Dự đoán hoàn tất.")

    print("\n  --- Kết quả đánh giá ---")
    h_loss = hamming_loss(y_test, y_pred)
    subset_acc = accuracy_score(y_test, y_pred)
    print(f"  Hamming Loss: {h_loss:.4f}")
    print(f"  Subset Accuracy: {subset_acc:.4f}")

    print("\n  Classification Report:")
    report = classification_report(
        y_test,
        y_pred,
        labels=np.arange(len(labels_list)), # Cung cấp chỉ số của các lớp
        target_names=labels_list,          # Tên tương ứng của các lớp
        zero_division=0,
        digits=4
    )
    print(report)
    print(f"--- Kết thúc {task_name} ---")
    return pipeline 


input_json_file = 'output_asca_format_corrected.json'
raw_data = load_asca_data(input_json_file)

if not raw_data:
    print("Không có dữ liệu để xử lý.")
else:
    print(f"Đã đọc {len(raw_data)} câu từ file JSON.")
    X_texts = [item['text'] for item in raw_data]
    y_categories_raw = [
        list(set(cat_pol['category'] for cat_pol in item.get('aspectCategories', []) if cat_pol.get('category')))
        for item in raw_data
    ]
    mlb_category = MultiLabelBinarizer()
    y_categories_binarized = mlb_category.fit_transform(y_categories_raw)
    category_labels = mlb_category.classes_
    y_polarities_raw = [
        list(set(cat_pol['polarity'] for cat_pol in item.get('aspectCategories', []) if cat_pol.get('polarity')))
        for item in raw_data
    ]
    mlb_polarity = MultiLabelBinarizer()
    y_polarities_binarized = mlb_polarity.fit_transform(y_polarities_raw)
    polarity_labels = sorted([p for p in ['positive', 'negative', 'neutral'] if p in mlb_polarity.classes_])
    y_catpol_raw = [
        [f"{cat_pol['category']}#{cat_pol['polarity']}"
         for cat_pol in item.get('aspectCategories', []) if cat_pol.get('category') and cat_pol.get('polarity')]
        for item in raw_data
    ]
    mlb_catpol = MultiLabelBinarizer()
    y_catpol_binarized = mlb_catpol.fit_transform(y_catpol_raw)
    catpol_labels = mlb_catpol.classes_
    indices = np.arange(len(X_texts))
    X_train_texts, X_test_texts, y_train_cat, y_test_cat, y_train_pol, y_test_pol, y_train_cp, y_test_cp, train_indices, test_indices = train_test_split(
        X_texts,
        y_categories_binarized,
        y_polarities_binarized,
        y_catpol_binarized,
        indices,
        test_size=0.2,
        random_state=42
    )

    print("-" * 30)
    print(f"Tổng số mẫu: {len(X_texts)}")
    print(f"Kích thước tập huấn luyện: {len(X_train_texts)}")
    print(f"Kích thước tập kiểm thử: {len(X_test_texts)}")
    print("-" * 30)

    print("\n" + "="*20 + " Tác vụ 1: Aspect Category Detection (ACD) " + "="*20)
    print(f"Số lượng Category duy nhất: {len(category_labels)}")
    pipeline_acd = train_evaluate_multilabel(X_train_texts, y_train_cat, X_test_texts, y_test_cat, category_labels, "ACD")

    print("\n" + "="*20 + " Tác vụ 2: Sentiment Polarity Classification (SPC) " + "="*20)
    print(f"Số lượng Polarity duy nhất: {len(polarity_labels)}")
    pipeline_spc = train_evaluate_multilabel(X_train_texts, y_train_pol, X_test_texts, y_test_pol, polarity_labels, "SPC")

    print("\n" + "="*20 + " Tác vụ 3: Category#Polarity Pair Detection (ASCA) " + "="*20)
    print(f"Số lượng Cặp Category#Polarity duy nhất: {len(catpol_labels)}")
    pipeline_catpol = train_evaluate_multilabel(X_train_texts, y_train_cp, X_test_texts, y_test_cp, catpol_labels, "ASCA Pairs")


    print("\nHoàn thành tất cả các tác vụ.")

Đã đọc 15519 câu từ file JSON.
------------------------------
Tổng số mẫu: 15519
Kích thước tập huấn luyện: 12415
Kích thước tập kiểm thử: 3104
------------------------------

==================== Tác vụ 1: Aspect Category Detection (ACD) ====================
Số lượng Category duy nhất: 8

--- Bắt đầu ACD ---
  Bắt đầu huấn luyện...
  Huấn luyện hoàn tất.
  Bắt đầu dự đoán...
  Dự đoán hoàn tất.

  --- Kết quả đánh giá ---
  Hamming Loss: 0.1089
  Subset Accuracy: 0.4501

  Classification Report:
                             precision    recall  f1-score   support

         Course information     0.4978    0.6799    0.5747       328
             General review     0.2922    0.4623    0.3581       292
       Learning environment     0.8125    0.7944    0.8034       180
Organization and management     0.5552    0.6598    0.6030       244
     Support from lecturers     0.5751    0.6744    0.6208       857
           Teaching quality     0.7376    0.7644    0.7508      1269
        Test a

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
# Import đầy đủ các hàm metrics
from sklearn.metrics import classification_report, accuracy_score, f1_score
import re

# --- Configuration ---
FILE_PATH = 'combined_cleaned_file.txt' # Đường dẫn tới file dữ liệu của bạn
TEST_SIZE = 0.2 # Tỷ lệ dữ liệu dùng để kiểm thử (validation)
RANDOM_STATE = 42 # Để đảm bảo kết quả lặp lại được

# --- Load Data ---
try:
    df = pd.read_csv(FILE_PATH, sep='\t', encoding='utf-8')
    print("Đã tải dữ liệu thành công.")
except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file tại đường dẫn '{FILE_PATH}'")
    exit()
except Exception as e:
    print(f"Lỗi khi đọc file: {e}")
    exit()

# --- Data Cleaning and Preparation ---
required_columns = ['Sentence Component', 'aspect', 'sentiment_text', 'sentiment']
df.dropna(subset=required_columns, inplace=True)
print(f"\nSố lượng dòng sau khi xóa NaN: {len(df)}")

# Loại bỏ các ký tự đặc biệt (Tùy chọn)
def clean_text(text):
    if isinstance(text, str):
        text = re.sub(r'colonsmilesmile|wzjwz\d+', '', text, flags=re.IGNORECASE)
        text = text.strip()
    return text
# df['Sentence Component'] = df['Sentence Component'].apply(clean_text)
# df['sentiment_text'] = df['sentiment_text'].apply(clean_text)

# --- Split Data into Training and Testing Sets (Using DataFrame) ---
train_df, test_df = train_test_split(
    df,
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
    stratify=df['aspect'] # Stratify theo aspect
)
print(f"\nSố lượng mẫu huấn luyện: {len(train_df)}")
print(f"Số lượng mẫu kiểm thử: {len(test_df)}")


# --- 1. Aspect Identification Model ---
print("\n--- Huấn luyện & Đánh giá Mô hình Nhận diện Aspect ---")
aspect_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 2))),
    ('clf', LinearSVC(random_state=42, dual="auto", class_weight='balanced'))
])
print("Đang huấn luyện mô hình Aspect...")
aspect_pipeline.fit(train_df['Sentence Component'] + train_df['Review'], train_df['aspect'])
print("Dự đoán Aspect trên tập kiểm thử (test_df)...")
predicted_aspects = aspect_pipeline.predict(test_df['Sentence Component'])
test_df['predicted_aspect'] = predicted_aspects
accuracy_aspect = accuracy_score(test_df['aspect'], test_df['predicted_aspect'])
print(f"Độ chính xác (Accuracy) riêng cho Aspect: {accuracy_aspect:.4f}")

# --- Hiển thị lại Classification Report cho Aspect ---
print("\nBáo cáo phân loại chi tiết cho Aspect:")
# Lấy danh sách các nhãn aspect duy nhất từ cả tập thực tế và dự đoán
all_aspect_labels = sorted(list(set(test_df['aspect']) | set(test_df['predicted_aspect'])))
# In báo cáo
print(classification_report(test_df['aspect'], test_df['predicted_aspect'], labels=all_aspect_labels, zero_division=0, digits=4))
# --- Kết thúc phần hiển thị lại ---


# --- 2. Sentiment Classification Model ---
print("\n--- Huấn luyện & Đánh giá Mô hình Phân loại Sentiment ---")
sentiment_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 2))),
    ('clf', LogisticRegression(solver='liblinear', multi_class='auto', random_state=RANDOM_STATE))
])
print("Đang huấn luyện mô hình Sentiment...")
sentiment_pipeline.fit(train_df['sentiment_text'] + train_df['Review'], train_df['sentiment'])
print("Dự đoán Sentiment trên tập kiểm thử (test_df)...")
predicted_sentiments = sentiment_pipeline.predict(test_df['sentiment_text'])
test_df['predicted_sentiment'] = predicted_sentiments
accuracy_sentiment = accuracy_score(test_df['sentiment'], test_df['predicted_sentiment'])
print(f"Độ chính xác (Accuracy) riêng cho Sentiment: {accuracy_sentiment:.4f}")

# --- Hiển thị lại Classification Report cho Sentiment ---
print("\nBáo cáo phân loại chi tiết cho Sentiment:")
# Lấy danh sách các nhãn sentiment duy nhất từ cả tập thực tế và dự đoán
all_sentiment_labels = sorted(list(set(test_df['sentiment']) | set(test_df['predicted_sentiment'])))
# In báo cáo
print(classification_report(test_df['sentiment'], test_df['predicted_sentiment'], labels=all_sentiment_labels, zero_division=0, digits=4))
# --- Kết thúc phần hiển thị lại ---


# --- 3. Combine Predictions and Evaluate Pair Performance ---
print("\n--- Đánh giá hiệu suất dự đoán CẶP {Aspect: Sentiment} ---")

test_df['true_pair'] = test_df.apply(lambda row: f"{{{row['aspect']}: {row['sentiment']}}}", axis=1)
test_df['predicted_pair'] = test_df.apply(lambda row: f"{{{row['predicted_aspect']}: {row['predicted_sentiment']}}}", axis=1)

pair_accuracy = accuracy_score(test_df['true_pair'], test_df['predicted_pair'])
print(f"Độ chính xác (Accuracy) cho việc dự đoán đúng cả cặp {{Aspect: Sentiment}}: {pair_accuracy:.4f}")

pair_f1_weighted = f1_score(test_df['true_pair'], test_df['predicted_pair'], average='weighted', zero_division=0)
pair_f1_macro = f1_score(test_df['true_pair'], test_df['predicted_pair'], average='macro', zero_division=0)
print(f"F1-score (Weighted) cho việc dự đoán đúng cả cặp {{Aspect: Sentiment}}: {pair_f1_weighted:.4f}")
print(f"F1-score (Macro)   cho việc dự đoán đúng cả cặp {{Aspect: Sentiment}}: {pair_f1_macro:.4f}")

print("\nBáo cáo phân loại chi tiết cho từng cặp {Aspect: Sentiment}:")
all_pair_labels = sorted(list(set(test_df['true_pair']) | set(test_df['predicted_pair'])))
print(classification_report(test_df['true_pair'], test_df['predicted_pair'], labels=all_pair_labels, zero_division=0, digits=4))


# --- 4. Display Predicted Aspect:Sentiment Pairs for Test Set (Example) ---
print("\n--- Hiển thị ví dụ Kết quả Dự đoán Cặp {Aspect: Sentiment} trên Tập Kiểm Thử ---")
print("(Hiển thị 20 mẫu đầu tiên của tập kiểm thử)")

columns_to_show = [
    'Sentence Component',
    'sentiment_text',
    'true_pair',
    'predicted_pair'
]
print(test_df[columns_to_show].head(20).to_string())

print("\n--- Kết thúc ---")

Đã tải dữ liệu thành công.

Số lượng dòng sau khi xóa NaN: 20779

Số lượng mẫu huấn luyện: 16623
Số lượng mẫu kiểm thử: 4156

--- Huấn luyện & Đánh giá Mô hình Nhận diện Aspect ---
Đang huấn luyện mô hình Aspect...
Dự đoán Aspect trên tập kiểm thử (test_df)...
Độ chính xác (Accuracy) riêng cho Aspect: 0.6939

Báo cáo phân loại chi tiết cho Aspect:
                             precision    recall  f1-score   support

         Course information     0.5936    0.7146    0.6485       417
             General review     0.4549    0.4242    0.4390       297
       Learning environment     0.7795    0.8879    0.8302       223
Organization and management     0.6176    0.7243    0.6667       301
     Support from lecturers     0.7285    0.6753    0.7009      1152
           Teaching quality     0.7753    0.7113    0.7419      1455
        Test and evaluation     0.6541    0.7591    0.7027       137
                   Workload     0.6106    0.7299    0.6649       174

                   accuracy

/home/duycute/.pyenv/versions/3.11.11/envs/Ai_ENV/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Dự đoán Sentiment trên tập kiểm thử (test_df)...
Độ chính xác (Accuracy) riêng cho Sentiment: 0.8590

Báo cáo phân loại chi tiết cho Sentiment:
              precision    recall  f1-score   support

    Negative     0.8407    0.8681    0.8542      1198
     Neutral     0.7143    0.5693    0.6336       606
    Positive     0.8970    0.9290    0.9127      2352

    accuracy                         0.8590      4156
   macro avg     0.8173    0.7888    0.8002      4156
weighted avg     0.8541    0.8590    0.8551      4156


--- Đánh giá hiệu suất dự đoán CẶP {Aspect: Sentiment} ---
Độ chính xác (Accuracy) cho việc dự đoán đúng cả cặp {Aspect: Sentiment}: 0.6011
F1-score (Weighted) cho việc dự đoán đúng cả cặp {Aspect: Sentiment}: 0.6022
F1-score (Macro)   cho việc dự đoán đúng cả cặp {Aspect: Sentiment}: 0.4881

Báo cáo phân loại chi tiết cho từng cặp {Aspect: Sentiment}:
                                         precision    recall  f1-score   support

         {Course information: Negativ

In [16]:
import json
import underthesea # For Vietnamese word tokenization
from sklearn.model_selection import train_test_split
import sklearn_crfsuite
from sklearn_crfsuite import metrics
import joblib

# --- 1. Load Data ---

def load_data(filepath):
    """Loads data from the JSON file."""
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            data = json.load(f)
        # Adjust based on the actual structure
        if "sentences" in data and "sentence" in data["sentences"]:
             return data["sentences"]["sentence"]
        elif isinstance(data, list): # Handle if data is directly a list of sentences
             return data
        else:
             raise ValueError("Cannot find sentence list in the JSON structure")
    except FileNotFoundError:
        print(f"Error: File not found at {filepath}")
        return []
    except json.JSONDecodeError:
        print(f"Error: Could not decode JSON from {filepath}")
        return []
    except Exception as e:
        print(f"An unexpected error occurred loading data: {e}")
        return []

# --- 2. Data Preparation ---

def word_tokenize(sentence):
  """Tokenizes a Vietnamese sentence."""
  # return sentence.split() # Simple whitespace split (less accurate)
  return underthesea.word_tokenize(sentence)

def get_bio_tags(tokens, aspects):
    """
    Generates BIO tags for a tokenized sentence based on aspect spans.
    Assumes non-overlapping aspects. Encodes category in the tag.
    """
    tags = ['O'] * len(tokens)
    token_spans = []
    current_pos = 0
    # Calculate character spans for each token
    for token in tokens:
        start = current_pos
        end = start + len(token)
        token_spans.append((start, end))
        # Assuming space separation after tokenization for simplicity
        current_pos = end + 1 # Move past the token and the assumed space

    for aspect in aspects:
        try:
            aspect_start = int(aspect['from'])
            aspect_end = int(aspect['to']) # Assumes exclusive
            category = aspect['category'].strip().replace(" ", "_") # Sanitize category name
            if not category: continue

            b_tag = f"B-{category}"
            i_tag = f"I-{category}"
            first_token_in_span = True

            for i, (tok_start, tok_end) in enumerate(token_spans):
                 # Check if token overlaps with the aspect span
                 # A simple overlap check: token starts within aspect OR aspect starts within token
                token_overlaps = (tok_start >= aspect_start and tok_start < aspect_end) or \
                                 (aspect_start >= tok_start and aspect_start < tok_end)

                if token_overlaps:
                     # Ensure we don't overwrite tags from other aspects (shouldn't happen with no_overlap file)
                    if tags[i] == 'O':
                        if first_token_in_span:
                            tags[i] = b_tag
                            first_token_in_span = False
                        else:
                            tags[i] = i_tag
        except (ValueError, KeyError) as e:
            print(f"Warning: Skipping aspect due to data issue: {aspect}. Error: {e}")

    return tags


# --- 3. Feature Extraction ---

def word2features(sent, i):
    """Extracts features for a word at position i in the sentence (list of tokens)."""
    word = sent[i]
    # Add more features here for better performance!
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        # You can add prefix/suffix features:
        'word.suffix(3)': word[-3:],
        'word.prefix(3)': word[:3],
         # Add shape features (e.g., 'Xxxx', 'dd'):
        'word.shape': ''.join(['X' if c.isupper() else 'x' if c.islower() else 'd' if c.isdigit() else c for c in word]),

    }
    # Features for previous word (if not beginning of sentence)
    if i > 0:
        word1 = sent[i-1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
        })
    else:
        features['BOS'] = True # Indicate Beginning Of Sentence

    # Features for next word (if not end of sentence)
    if i < len(sent)-1:
        word1 = sent[i+1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
        })
    else:
        features['EOS'] = True # Indicate End Of Sentence

    # --- Potential improvements (require more libraries/effort): ---
    # - Part-of-Speech (POS) tags for current, previous, next words
    # - Word embedding features (e.g., from PhoBERT, Word2Vec)
    # - Gazetteer features (is the word in a predefined list of locations, names, etc.?)

    return features

def sent2features(sent):
    """Applies word2features to each word in the sentence."""
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(tags):
    """Returns the list of labels (already prepared)."""
    return tags

# --- Main Data Processing ---

input_file = 'output_semeval_format_v3_no_overlap.txt'
raw_data = load_data(input_file)

if not raw_data:
    print("Exiting due to data loading errors.")
    exit()

print(f"Loaded {len(raw_data)} sentences.")

# Process all sentences
X = [] # List of feature dict sequences for sentences
y = [] # List of label sequences for sentences

print("Processing sentences and extracting features...")
for sentence_data in raw_data:
    if 'text' not in sentence_data or 'aspects' not in sentence_data:
        continue
    text = sentence_data['text']
    aspects = sentence_data['aspects']

    tokens = word_tokenize(text)
    if not tokens: # Skip empty sentences after tokenization
         continue

    tags = get_bio_tags(tokens, aspects)
    features = sent2features(tokens)

    # Ensure features and tags have the same length
    if len(features) == len(tags):
        X.append(features)
        y.append(tags)
    else:
        print(f"Warning: Mismatch length between features ({len(features)}) and tags ({len(tags)}) for sentence: '{text}'. Skipping.")


print(f"Processed {len(X)} sentences successfully.")

if not X or not y:
     print("No data available for training after processing. Exiting.")
     exit()

# --- 4. Train/Test Split ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

print(f"\nTrain size: {len(X_train)}")
print(f"Test size: {len(X_test)}")

# --- 5. Initialize and Train CRF Model ---
print("\nInitializing and training CRF model...")

# Define CRF model with hyperparameters
# You might need to tune these hyperparameters (c1, c2) using cross-validation
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', # Limited-memory Broyden–Fletcher–Goldfarb–Shanno algorithm
    c1=0.1,            # Coefficient for L1 penalty
    c2=0.1,            # Coefficient for L2 penalty
    max_iterations=100,# Stop fitting after some iterations
    all_possible_transitions=True # Generate transitions for all label pairs
)

# Train the model
try:
    crf.fit(X_train, y_train)
    print("Training complete.")
except Exception as e:
    print(f"An error occurred during training: {e}")
    # Potentially print shapes or first elements of X_train/y_train for debugging
    # print("X_train[0]:", X_train[0] if X_train else "Empty")
    # print("y_train[0]:", y_train[0] if y_train else "Empty")
    exit()


# --- 6. Evaluation ---
print("\nEvaluating model on test set...")

# Get all unique labels, excluding 'O' for aggregated metrics if needed
labels = list(crf.classes_)
# print("Model Classes (Labels):", labels)

# Predict on test set
y_pred = crf.predict(X_test)

# Calculate metrics (Precision, Recall, F1)
# Use flat_classification_report for sequence labeling tasks
# Filter out 'O' tag for a more meaningful F1 score for aspect terms
sorted_labels = sorted(
    [label for label in labels if label != 'O'],
    key=lambda name: (name[1:], name[0]) # Sort primarily by category name, then by B/I
)

try:
    report = metrics.flat_classification_report(
        y_test, y_pred, labels=sorted_labels, digits=3
    )
    print("\nFlat Classification Report (excluding 'O'):")
    print(report)
except Exception as e:
     print(f"Error generating classification report: {e}")
     # Fallback or print raw predictions/labels for debugging
     # print("Sample True Labels:", y_test[0] if y_test else "N/A")
     # print("Sample Predicted Labels:", y_pred[0] if y_pred else "N/A")


# --- 7. Save the Model (Optional) ---
model_filename_crf = 'sklearn_crf_aspect_extractor.joblib'
joblib.dump(crf, model_filename_crf)
print(f"\nĐã lưu mô hình CRF vào file: {model_filename_crf}")


# --- 8. Prediction Example ---
# Load model: loaded_crf = joblib.load(model_filename_crf)

print("\n--- Ví dụ dự đoán Aspect ---")
test_sentence = "giáo viên nhiệt tình nhưng cơ sở vật chất cần cải thiện ."
test_tokens = word_tokenize(test_sentence)
test_features = sent2features(test_tokens)

predicted_tags = crf.predict_single(test_features)

print(f"Câu: {test_sentence}")
print("Tokens:", test_tokens)
print("Predicted BIO Tags:", predicted_tags)

# Optional: Group tags into spans
def group_tags(tokens, tags):
    extracted_aspects = []
    current_aspect_tokens = []
    current_aspect_type = None
    for token, tag in zip(tokens, tags):
        if tag.startswith("B-"):
            if current_aspect_tokens: # Finish previous aspect
                extracted_aspects.append({"term": " ".join(current_aspect_tokens), "category": current_aspect_type})
            current_aspect_tokens = [token]
            current_aspect_type = tag.split("-", 1)[1]
        elif tag.startswith("I-"):
            tag_type = tag.split("-", 1)[1]
            if current_aspect_tokens and tag_type == current_aspect_type: # Continue current aspect
                current_aspect_tokens.append(token)
            else: # I- tag without B- or mismatch type - treat as O or reset
                 if current_aspect_tokens: # Finish previous aspect
                      extracted_aspects.append({"term": " ".join(current_aspect_tokens), "category": current_aspect_type})
                 current_aspect_tokens = []
                 current_aspect_type = None
        else: # 'O' tag
            if current_aspect_tokens: # Finish previous aspect
                extracted_aspects.append({"term": " ".join(current_aspect_tokens), "category": current_aspect_type})
            current_aspect_tokens = []
            current_aspect_type = None
    # Add the last aspect if any
    if current_aspect_tokens:
        extracted_aspects.append({"term": " ".join(current_aspect_tokens), "category": current_aspect_type})
    return extracted_aspects

extracted = group_tags(test_tokens, predicted_tags)
print("Extracted Aspects:", extracted)

Loaded 15519 sentences.
Processing sentences and extracting features...
Processed 15519 sentences successfully.

Train size: 11639
Test size: 3880

Initializing and training CRF model...
Training complete.

Evaluating model on test set...

Flat Classification Report (excluding 'O'):
                               precision    recall  f1-score   support

         B-Course_information      0.542     0.379     0.446       428
         I-Course_information      0.270     0.274     0.272       862
             B-General_review      0.349     0.202     0.256       361
             I-General_review      0.173     0.197     0.184      1398
       B-Learning_environment      0.741     0.564     0.640       243
       I-Learning_environment      0.621     0.438     0.513       592
B-Organization_and_management      0.367     0.238     0.289       340
I-Organization_and_management      0.367     0.224     0.278      1595
     B-Support_from_lecturers      0.428     0.282     0.340      1143
     